In [8]:
Zx.<x> = ZZ[]
def convolution(f,g,n):
      return (f * g) % (x^n-1)

def balancedmod(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return Zx(g)

def balancedmod_ar(ar,q):
    return (ar+q/2)%q-q/2

def rbalancedmod_ar(ar,q):
    return (ar+q//2)%q-q//2



def randomdpoly(n,d):
    assert d <= n
    result = n*[0]
    for j in range(d):
        while True:
            r = randrange(n)
            #print(r)
            if not result[r]: break
        result[r] = 1-2*randrange(2)
    return Zx(result)


def randomdpolyq(n,d,q):
    assert d<=n
    assert q%2==0
    
    result=n*[0]
    
    for j in range(d):
        result[j]= q/2-randrange(q)-1
        
    p = Permutations(len(result)).random_element() 
    result_rand = [result[p[i]-1] for i in range(len(result))]
    out=Zx(result_rand)
    
    return out



def invertmodprime(f,p,n):
    T = Zx.change_ring(Integers(p)).quotient(x^n-1)
    return Zx(lift(1 / T(f)))



def invertmodpowerof2(f,q,n):
    assert q.is_power_of(2)
    g = invertmodprime(f,2,n)
    while True:
        r = balancedmod(convolution(g,f,n),q,n)
        if r == 1: return g
        g = balancedmod(convolution(g,2 - r,n),q,n)
            
        

def keypair(n,d,q):
    while True:
        try:
            f = randomdpoly(n,d)
            f3 = invertmodprime(f,3,n)
            fq = invertmodpowerof2(f,q,n)
            break
        except:
            pass
    g = randomdpoly(n,d)
    #print(g)
    publickey = balancedmod(3 * convolution(fq,g,n),q,n)
    secretkey = f,f3
    return publickey,secretkey


def randommessage(n):
    result = list(randrange(3)-1 for j in range(n))
    return Zx(result)

def encrypt(message,publickey,n,d):
    r = randomdpoly(n,d)
    return balancedmod(convolution(publickey,r,n) + message,q,n)

def decrypt(ciphertext,secretkey):
    f,f3 = secretkey
    a = balancedmod(convolution(ciphertext,f,n),q,n)
    return balancedmod(convolution(a,f3,n),3,n)


import numpy as np

def T(f):
    temp=np.zeros((len(f), len(f)))
    for i in range(len(f)):
        temp[i]=np.roll(f,i)
        
    return temp

def That(f):
    temp=np.zeros((len(f), len(f)))
    for i in range(len(f)):
        temp[i]=np.roll(f,-i)
        
    return temp

def sigma(f):
    temp=np.zeros(len(f))
    temp[0]=f[0]
    temp[1:]=f[1:][::-1]
    
    return temp

def Tsigma(f):
    temp=np.zeros((len(f), len(f)))
    for i in range(len(f)):
        temp[i]=sigma(np.roll(f,i))
    
    return temp

def sec_basis(f,g,q,n,l):
    f_sigma=Tsigma(f)
    Tg=T(g)
    return np.sqrt(l/q)*np.block([[f_sigma,Tg]])#,[np.zeros((n,n)), q*np.eye(n)]])
    

def mat_keypair(n,d,q,p):
    for i in range(int(1e7)):
        try:
            f=1+p*randomdpoly(n,d-1)
            fq=invertmodpowerof2(f,q,n)
            g = p*randomdpoly(n,d)
            publickey = balancedmod(convolution(fq,g,n),q,n)
            
            public_inv=invertmodpowerof2(publickey,q,n)
            print("SUCCESS")
            break
        except:
            pass
    
    
    h=np.asarray([publickey[i] for i in range(n)])
    h_inv=np.asarray([public_inv[i] for i in range(n)])
    f_vec=np.asarray([f[i] for i in range(n)])
    g_vec=np.asarray([g[i] for i in range(n)])
    
    pk=That(h)
    sk=Tsigma(f_vec)
    pk_inv=Tsigma(h_inv)
    

    
    return h,h_inv,f_vec, g_vec


def pub_basis(pk,q,n, l):
    return np.sqrt(l/q)*np.block([[np.eye(n),pk],[np.zeros((n,n)), q*np.eye(n)]])


def randomstring(n):
    return 1-np.random.randint(3,size=n)

def cipher(m,r,pk,n,q):
    return balancedmod_ar(m+pk@r,q)


def Jsym(n):
    J1=np.asarray([[0,1],[-1,0]])
    J=np.kron(J1,np.eye(n))
    return J

def NTRUlattice(publickey,n,d,q,p):
    #n,d,q=n,d,q
    #publickey,secretkey = keypair(n,d,q)
    recipp = lift(1/Integers(q)(p))
    publickeyoverp = balancedmod(recipp * publickey,q,n)
    
    M = matrix(2 * n)
    for i in range(n,2*n):
        M[i,i] = q
    for i in range(n):
        M[i,i] = 1
        c = convolution(x^(n-i),publickeyoverp,n)
        for j in range(n):
            M[i,j+n] = c[j]
    
    return M


from sage.modules.free_module_integer import IntegerLattice

def NTRU_len(publickey,n,d,q,p):
    M=NTRUlattice(publickey,n,d,q,p)
    L=IntegerLattice(M)
    
    SVP=L.shortest_vector()#algorithm="pari")
    
    return SVP.norm().n()

def NTRU_len(publickey,n,d,q,p):
    M=NTRUlattice(publickey,n,d,q,p)
    L=IntegerLattice(M)
    
    L.BKZ(block_size=n)
    #SVP=L.shortest_vector()#algorithm="pari")
    
    return min(v.norm().n() for v in L.reduced_basis)#SVP.norm().n()

def sNTRU_len(f,g,n):
    f_sigma=Tsigma(f)
    Tg=T(g)
    s_bas=np.block([[f_sigma,Tg]])
    
    L=IntegerLattice(s_bas)
    L.LLL()
    #L.BKZ(block_size=np.floor(n/2))
    #SVP=min(np.linalg.norm(v) for v in s_bas)
    SVP=min(v.norm().n() for v in L.reduced_basis)
    #SVP=L.shortest_vector().norm().n()
    #SVP=L.shortest_vector(algorithm="pari").norm().n()
    
    return SVP




NUM=10

Ps=[3,5,7,15,21]
Qs=[4,8,16,32,64]
Ns=[7,11,17,53, 97]
Ds=[5,7,11,29,53]
opt_len=[0]*len(Ns)
for i in [-2,-1]:#range(len(Ns)):
    q=Qs[i]
    p=Ps[i]
    n=Ns[i]
    d=Ds[i]
    print("n,d=", n,d)
    temp=[]
    temp_len=[0]*NUM
    for _ in range(NUM):
        print(_)
        h,h_inv,f_vec, g_vec=mat_keypair(n,d,q,p)
        tl=sNTRU_len(f_vec,g_vec,n)#NTRU_len(h,n,d,q,p)
        temp_len[_]=tl

        temp+=[[h,h_inv,f_vec, g_vec]]
        
    indmax=np.argmax(temp_len)
    opt_len[i]=temp_len[indmax]
    optimal=np.asarray(temp[indmax])
    #print(optimal[0].shape)
    print("opt_l=%.3f"%(temp_len[indmax]))
    
    np.savetxt("NTRU_opt_n%i_q%i_p%i_d%i.txt"%(n,q,p,d), optimal,comments="#sLLL_lambda_1=%.2f"%(temp_len[indmax]))
    print("n,d=", n,d, "DONE")
    
    
#import matplotlib.pyplot as plt
#plt.plot(Ns, opt_len)
#plt.show()


#opt_params=np.asarray([[Ns[i], Ds[i], opt_len[i]] for i in range(len(Ns))])
#np.savetxt("NTRU_parameters_q%i_p%i_NUM%i.txt"%(q,p,NUM), opt_params)

n,d= 53 29
0
SUCCESS
1
SUCCESS
2
SUCCESS
3
SUCCESS
4
SUCCESS
5
SUCCESS
6
SUCCESS
7
SUCCESS
8
SUCCESS
9
SUCCESS
opt_l=113.384
n,d= 53 29 DONE
n,d= 97 53
0
SUCCESS
1
SUCCESS
2
SUCCESS
3
SUCCESS
4
SUCCESS
5
SUCCESS
6
SUCCESS
7
SUCCESS
8
SUCCESS
9
SUCCESS
opt_l=215.286
n,d= 97 53 DONE


In [9]:
temp_len[indmax]

215.285856479240

In [ ]:
pk,sk,pinv=That(h),Tsigma(f_vec), Tsigma(h_inv)

In [ ]:
rbalancedmod_ar(sk,5)

In [ ]:
rbalancedmod_ar(rbalancedmod_ar((sk@pk),256),5)

In [ ]:
n = 7
d = 5
q = 256

for i in range(int(1e7)):
    try:
        f=1+7*randomdpoly(n,d-1)
        fq=invertmodpowerof2(f,q,n)
        print("SUCCESS")
        print(fq)
        break
    except:
        pass

print(f)
#invertmodpowerof2(f,q,n)

In [ ]:
h,h_inv,f_vec, g_vec=mat_keypair(n,d,q,7)
pk,sk,pinv=That(h),Tsigma(f_vec), Tsigma(h_inv)

In [ ]:
pk

In [ ]:
pinv

In [ ]:
sig_check=balancedmod_ar(pinv@pk,q)
print(sig_check)

In [ ]:
sk@pk

In [ ]:
balancedmod_ar(sk@pk, q)

In [ ]:
def randomstring(n):
    return 1-np.random.randint(3,size=n)

def cipher(m,r,pk,n,q):
    return balancedmod_ar(m+pk@r,q)

m=randomstring(n)
r=randomstring(n)
print("m")
print(m)
print("r")
print(r)
c=cipher(m,r,pk,n,q)
print("c")
print(c)

In [ ]:
a=balancedmod_ar(sk@c,q)
print(a)

In [ ]:
amodp=balancedmod_ar(a,7)
print(amodp)
print(np.all(amodp==m))

In [ ]:
c-amodp

In [ ]:
b=pinv@(c-amodp)
print(np.all(sigma(balancedmod_ar(b,q))==r))

In [ ]:
def Jsym(n):
    J1=np.asarray([[0,1],[-1,0]])
    J=np.kron(J1,np.eye(n))
    return J

M=pub_basis(pk,q,n, 2)



In [ ]:
##test
n = 7
d = 5
q = 256
for tests in range(10):
    publickey,secretkey = keypair(n,d,q)
    m = randommessage(n)
    c = encrypt(m,publickey,n,d)
    print(c)
    print(m== decrypt(c,secretkey))
    
def attack(publickey):
    recip3 = lift(1/Integers(q)(3))
    publickeyover3 = balancedmod(recip3 * publickey,q,n)
    M = matrix(2 * n)
    for i in range(n):
        M[i,i] = q
    for i in range(n):
        M[i+n,i+n] = 1
        c = convolution(x^i,publickeyover3,n)
        for j in range(n):
            M[i+n,j] = c[j]
    M = M.LLL()
    for j in range(2 * n):
        try:
            f = Zx(list(M[j][n:]))
            f3 = invertmodprime(f,3,n)
            return (f,f3)
        except:
            pass
        return (f,f)
    
    
def NTRUlattice(publickey,n,d,q,p):
    #n,d,q=n,d,q
    #publickey,secretkey = keypair(n,d,q)
    recipp = lift(1/Integers(q)(p))
    publickeyoverp = balancedmod(recipp * publickey,q,n)
    
    M = matrix(2 * n)
    for i in range(n,2*n):
        M[i,i] = q
    for i in range(n):
        M[i,i] = 1
        c = convolution(x^(n-i),publickeyoverp,n)
        for j in range(n):
            M[i,j+n] = c[j]
    
    return M


from sage.modules.free_module_integer import IntegerLattice

def NTRU_len(publickey,n,d,q,p):
    M=NTRUlattice(publickey,n,d,q,p)
    L=IntegerLattice(M)
    
    SVP=L.shortest_vector(algorithm="pari")
    
    return SVP.norm().n()

In [ ]:
NUM=8
q=256
d=4
p=7

Ns=[7,11,17,23,31]
Ds=[5,7,11,17,23]
opt_len=[0]*len(Ns)
for i in range(len(Ns)):
    n=Ns[i]
    d=Ds[i]
    print("n,d=", n,d)
    temp=[]
    temp_len=[0]*NUM
    for _ in range(NUM):
        print(_)
        h,h_inv,f_vec, g_vec=mat_keypair(n,d,q,p)
        #print("lh=",len(h))
        tl=NTRU_len(h,n,d,q,p)
        temp_len[_]=tl

        temp+=[[h,h_inv,f_vec, g_vec]]
        
    indmax=np.argmax(temp_len)
    opt_len[i]=temp_len[indmax]
    optimal=np.asarray(temp[indmax])
    #print(optimal[0].shape)
    
    np.savetxt("NTRU_opt_n%i_q%i_p%i_d%i.txt"%(n,q,p,d), optimal)
    print("n,d=", n,d, "DONE")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Ns, opt_len)
plt.show()


In [ ]:
opt_params=np.asarray([[Ns[i], Ds[i], opt_len[i]] for i in [-1]])
np.savetxt("NTRU_parameters_q%i_p%i_NUM%i.txt"%(q,p,NUM), opt_params)

In [ ]:
M=NTRUlattice(h,n,d,q)

In [ ]:
NTRU_len(h,n,d,q)

In [ ]:
L=IntegerLattice(M)

In [ ]:
L.shortest_vector(algorithm="pari").norm().n()

In [ ]:
np.sqrt(np.linalg.norm(f_vec)**2+np.linalg.norm(g_vec)**2)

In [ ]:
n = 37
d = 26
q = 256
M,s=randomNTRUlattice(n,d,q)

In [ ]:
M

In [ ]:
from sage.modules.free_module_integer import IntegerLattice
L=IntegerLattice(M, lll_reduce=False)

In [ ]:
min(v.norm().n() for v in L.reduced_basis)

In [ ]:
L.LLL()
min(v.norm().n() for v in L.reduced_basis)

In [ ]:
L.BKZ(block_size=n)
print(min(v.norm().n() for v in L.reduced_basis))
print(L.reduced_basis[0])
print(L.reduced_basis[0].norm().n())

In [ ]:
from fpylll import IntegerMatrix, SVP
#M=matrix.identity(7)
L = IntegerMatrix.from_matrix(M)
w = vector(ZZ, SVP.shortest_vector(L))


In [ ]:
L=IntegerLattice(M, lll_reduce=True)

In [ ]:
sage.crypto.gen_lattice(m=10, seed=42)

In [ ]:
from sage.modules.free_module_integer import IntegerLattice

temp=[]
for n in range(1,30):
    #m=2*round(log(n,2).n())+1
    q=2
    print(n,q)
    for num in range(20):
        set_random_seed(num*n)
        L=IntegerLattice(sage.crypto.gen_lattice(type='ideal',n=n,m=2*n, q=q, quotient=x^n-1))
        SV_len=L.shortest_vector(algorithm="pari").norm().n()
        temp+=[(n,SV_len)]

In [ ]:
for q in range(2,20):
    temp=[]
    nmax=10
    for n in range(2,nmax):
        print(n,q)
        for num in range(10):
            set_random_seed(num*n)
            L=IntegerLattice(sage.crypto.gen_lattice(type='ideal',n=n,m=2*n, q=q, quotient=x^n-1))
            SV_len=L.shortest_vector().norm().n()
            temp+=[(n,SV_len)]
            
    g=scatter_plot(temp, alpha=0.5, markersize=10, axes_labels=[r'$n$', r'$\lambda_1$'])+plot(sqrt(x*q/(pi*e)), (x,0,nmax))
    g=g+text(r'$\sqrt{\frac{n q}{\pi e}},\, q=%i$'%(q), ((nmax+1),sqrt((nmax+1)*q/(pi*e)+0.1)))
    g.show()
    g.save('NTRU_samples_q%i.png'%(q))

In [ ]:
temp

In [ ]:
g=scatter_plot(temp, alpha=0.5, markersize=10, axes_labels=['$n$', '$\lambda_1$'])+plot(sqrt(x*2/(pi*e)), (x,0,20))

In [ ]:
g=g+text("q=2", (22,sqrt(22*2/(pi*e)+0.1)))

In [ ]:
g.show()

In [ ]:
q

In [ ]:
round(log(17,2).n())